# Langchain Agent with custom tool
- Retrieve information from Weather API and use LLM to process the answer;
- Use Gradio as WebUI.

In [7]:
import os
from dotenv import load_dotenv
import gradio as gr
import requests
from typing import Dict, Any

In [8]:
# Langchain imports
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

### Setup model and load env variables

In [9]:
MODEL = 'gemini-2.0-flash'

# Load environment variables from .env file
load_dotenv(override=True)
GOOGLE_GEMINI_KEY = os.environ['GOOGLEAI_API_KEY']
WEATHER_API_KEY = os.environ['WEATHER_API_KEY'] # https://www.weatherapi.com/

### Setup tools

In [10]:
@tool
def get_weather(location: str) -> Dict[str, Any]:
    """Check the current weather conditions for a specific city."""
    url = f'http://api.weatherapi.com/v1/current.json?key={WEATHER_API_KEY}&q={location}&aqi=no'
    response = requests.get(url)
    if response.ok:
        data = response.json()
        # Return structured weather data
        return {
            'city': location,
            'temperature': data['current']['temp_c'],
            'unidade': 'Celsius',
            'condition': data['current']['condition']['text'],
            'humidity': data['current']['humidity'],
            'wind_kph': data['current']['gust_kph'],
            'is_day': data['current']['is_day'],
        }
    else:
        return f"Could not retrieve weather data for {location}."   

tools = [get_weather]

### Create agent (Gemini)

In [11]:
# Setup LLM model (Gemini)
llm = ChatGoogleGenerativeAI(
    model=MODEL,
    google_api_key=GOOGLE_GEMINI_KEY,
    temperature=0.7
)

# Agent prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a friendly and informative weather assistant.
    
    Your role is to help users get information about the weather in different cities.
      
    When the user asks about the weather in a city, use the available tool to query the weather data.
      
    IMPORTANT: The tool returns raw data in a structured format. You should create a natural and conversational response based on this data.
    Vary the format of your responses to make them more natural and personalized. Based on the weather, you can add relevant suggestions 
    (such as dressing appropriately, taking an umbrella, enjoying the sunny day, etc.). Use natural and friendly language.
      
    Examples of how you can vary your responses:
    - "It's 25°C in São Paulo today, with sunny skies and 60% humidity. A perfect day for outdoor activities!"
    - "In Lisbon right now: 18°C ​​with cloudy conditions. Humidity is at 75% and the wind is at 12km/h. You might want to bring a light jacket."
    - "The weather in New York is rainy right now. The temperature is 15°C with 85% humidity. Don't forget your umbrella!"
    
    Be polite, helpful, and provide information naturally."""),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

# Create the agent executor
agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
)

### Setup message history

In [12]:
# Setup message history
# This will store the chat history for each session
message_histories = {}

# The message history will be stored in a dictionary with session_id as the key
def get_session_history(session_id: str) -> ChatMessageHistory:
    if session_id not in message_histories:
        message_histories[session_id] = ChatMessageHistory()
    return message_histories[session_id]

# Configuring the agent executor to use message history
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

# Function for Gradio to process messages
def chat(message, history, session_id: str = None):
    response = agent_with_chat_history.invoke(
        {"input": message},
        config={"configurable": {"session_id": session_id}}
    )
    return response["output"]

# Prompt examples for the Gradio interface
examples = [
    ["How is the weather in São Paulo?", "default"],
    ["What is the temperature in Lisbon?", "default"],
    ["I need to know the weather in New York", "default"],
    ["Is it raining in Tokyo?", "default"]
]

### Start Gradio Chat UI

In [ ]:
# Start Gradio interface
demo = gr.ChatInterface(
    theme="soft",
    fn=chat,
    title="Weather Assistant",
    description="Ask about the weather in any city! (Example: 'What's the weather like in Buenos Aires?')",
    examples=examples,
    additional_inputs=[gr.Textbox(value="default", visible=True, label="Session ID")],
    type="messages"
).launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.




> Entering new AgentExecutor chain...

Invoking: `get_weather` with `{'location': 'São Paulo'}`


{'city': 'São Paulo', 'temperature': 27.3, 'unidade': 'Celsius', 'condition': 'Sunny', 'humidity': 54, 'wind_kph': 17.3, 'is_day': 1}It's 27.3°C in São Paulo today, with sunny skies and 54% humidity. The wind is at 17.3 km/h. A perfect day for outdoor activities!


> Finished chain.
